<a href="https://colab.research.google.com/github/shahzaib06042005/-Music-Genre-Classification/blob/main/Music_Genre_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# Copy the file from Drive to Colab workspace (optional but faster)
!cp "/content/drive/MyDrive/kaggle/fma_metadata.zip" .

# Unzip
!unzip -q fma_metadata.zip -d fma_metadata


In [5]:
!pip install librosa matplotlib scikit-learn tensorflow --quiet


In [9]:
import pandas as pd
import numpy as np

# Load precomputed features
features = pd.read_csv('fma_metadata/fma_metadata/features.csv', index_col=0, header=[0, 1, 2])
print("Features shape:", features.shape)

# Load genre labels
tracks = pd.read_csv('fma_metadata/fma_metadata/tracks.csv', index_col=0, header=[0, 1])
genre_labels = tracks['track']['genre_top']


Features shape: (106574, 518)


In [10]:
# Drop samples with missing genre
genre_labels = genre_labels.dropna()

# Filter features to include only these tracks
features = features.loc[genre_labels.index]

# Drop any remaining NaNs from features
features = features.dropna()

# Align genres again with filtered features
genre_labels = genre_labels.loc[features.index]

# Encode genre labels to integers
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(genre_labels)

print("Genre classes:", list(label_encoder.classes_))


Genre classes: ['Blues', 'Classical', 'Country', 'Easy Listening', 'Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental', 'International', 'Jazz', 'Old-Time / Historic', 'Pop', 'Rock', 'Soul-RnB', 'Spoken']


In [11]:
# Flatten multi-index columns
X = features.copy()
X.columns = ['_'.join(col).strip() for col in X.columns.values]
X = X.values  # convert to numpy

print("Final feature shape:", X.shape)


Final feature shape: (49598, 518)


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


Accuracy: 0.6265120967741935
                     precision    recall  f1-score   support

              Blues       0.00      0.00      0.00        22
          Classical       0.83      0.77      0.80       246
            Country       1.00      0.03      0.05        39
     Easy Listening       0.00      0.00      0.00         5
         Electronic       0.59      0.69      0.63      1874
       Experimental       0.53      0.72      0.61      2122
               Folk       0.61      0.51      0.56       561
            Hip-Hop       0.78      0.46      0.58       710
       Instrumental       0.61      0.15      0.24       416
      International       0.86      0.21      0.34       278
               Jazz       1.00      0.04      0.08       114
Old-Time / Historic       0.94      0.85      0.89       111
                Pop       0.36      0.01      0.02       466
               Rock       0.69      0.83      0.75      2836
           Soul-RnB       0.00      0.00      0.00     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
import joblib
joblib.dump(model, 'genre_rf_model.pkl')


['genre_rf_model.pkl']

In [14]:
from google.colab import files
files.download('genre_rf_model.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
import joblib

# Load the saved model
model = joblib.load('genre_rf_model.pkl')

# Use it to predict (example)
y_pred = model.predict(X_test)  # or new data
